In [5]:
import matplotlib.pyplot as plt
import fitbit

# gather_keys_oauth2.py file needs to be in the same directory. 
# also needs to install cherrypy: https://pypi.org/project/CherryPy/
#!pip install CherryPy
import gather_keys_oauth2 as Oauth2
import pandas as pd 
import datetime

import cherrypy

CLIENT_ID='238RWF'
CLIENT_SECRET='e1209868af85cb05ade8916a3382f33f'

server=Oauth2.OAuth2Server(CLIENT_ID, CLIENT_SECRET)
server.browser_authorize()
ACCESS_TOKEN=str(server.fitbit.client.session.token['access_token'])
REFRESH_TOKEN=str(server.fitbit.client.session.token['refresh_token'])
auth2_client=fitbit.Fitbit(CLIENT_ID,CLIENT_SECRET,oauth2=True,access_token=ACCESS_TOKEN,refresh_token=REFRESH_TOKEN)



[10/Sep/2022:00:33:42] ENGINE Listening for SIGTERM.
[10/Sep/2022:00:33:42] ENGINE Listening for SIGHUP.
[10/Sep/2022:00:33:42] ENGINE Listening for SIGUSR1.
[10/Sep/2022:00:33:42] ENGINE Bus STARTING
[10/Sep/2022:00:33:42] ENGINE Started monitor thread 'Autoreloader'.
[10/Sep/2022:00:33:42] ENGINE Serving on http://127.0.0.1:8080
[10/Sep/2022:00:33:42] ENGINE Bus STARTED


127.0.0.1 - - [10/Sep/2022:00:33:50] "GET /?code=9cf007b7d6fc7a19cb3c5e46a2ac8111a26fd448&state=qtdNWSj8GtTHCFTZtjbs9DcoScwh2o HTTP/1.1" 200 122 "" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.5 Safari/605.1.15"
127.0.0.1 - - [10/Sep/2022:00:33:50] "GET /favicon.ico HTTP/1.1" 200 1406 "http://127.0.0.1:8080/?code=9cf007b7d6fc7a19cb3c5e46a2ac8111a26fd448&state=qtdNWSj8GtTHCFTZtjbs9DcoScwh2o" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.5 Safari/605.1.15"


[10/Sep/2022:00:33:51] ENGINE Bus STOPPING
[10/Sep/2022:00:33:51] ENGINE HTTP Server cherrypy._cpwsgi_server.CPWSGIServer(('127.0.0.1', 8080)) shut down
[10/Sep/2022:00:33:51] ENGINE Stopped thread 'Autoreloader'.
[10/Sep/2022:00:33:51] ENGINE Bus STOPPED
[10/Sep/2022:00:33:51] ENGINE Bus EXITING
[10/Sep/2022:00:33:51] ENGINE Bus EXITED
[10/Sep/2022:00:33:51] ENGINE Waiting for child threads to terminate...


In [12]:
#기준날짜
oneDate = pd.datetime(year = 2022, month = 9, day = 6)

/Users/dayeong/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  


In [13]:
help(auth2_client.intraday_time_series)

Help on method intraday_time_series in module fitbit.api:

intraday_time_series(resource, base_date='today', detail_level='1min', start_time=None, end_time=None) method of fitbit.api.Fitbit instance
    The intraday time series extends the functionality of the regular time series, but returning data at a
    more granular level for a single day, defaulting to 1 minute intervals. To access this feature, one must
    fill out the Private Support form here (see https://dev.fitbit.com/docs/help/).
    For details on the resources available and more information on how to get access, see:
    
    https://dev.fitbit.com/docs/activity/#get-activity-intraday-time-series



In [14]:
#예시 1 - 심박수 뽑기
oneDayData = auth2_client.intraday_time_series('activities/heart',
                                               base_date=oneDate,
                                               detail_level='1sec')

In [15]:
oneDayData

{'activities-heart': [{'dateTime': '2022-09-06',
   'value': {'customHeartRateZones': [],
    'heartRateZones': [{'caloriesOut': 1288.08,
      'max': 115,
      'min': 30,
      'minutes': 1440,
      'name': 'Out of Range'},
     {'caloriesOut': 0,
      'max': 142,
      'min': 115,
      'minutes': 0,
      'name': 'Fat Burn'},
     {'caloriesOut': 0,
      'max': 177,
      'min': 142,
      'minutes': 0,
      'name': 'Cardio'},
     {'caloriesOut': 0,
      'max': 220,
      'min': 177,
      'minutes': 0,
      'name': 'Peak'}]}}]}

In [16]:
df = pd.DataFrame(oneDayData['activities-heart-intraday']['dataset'])

KeyError: 'activities-heart-intraday'

In [17]:
df.head()

NameError: name 'df' is not defined

In [10]:
#예시 2 - 칼로리 뽑기
dayDatacalories= auth2_client.intraday_time_series('activities/calories',base_date=oneDate,detail_level='1min')

In [11]:
d_calories = pd.DataFrame(dayDatacalories['activities-calories-intraday']['dataset'])

In [12]:
d_calories

,level,mets,time,value
0,0,10,00:00:00,1.06160
1,0,10,00:01:00,1.06160
2,0,10,00:02:00,1.06160
3,0,10,00:03:00,1.06160
4,0,10,00:04:00,1.06160
...,...,...,...,...
1435,0,11,23:55:00,1.16776
1436,0,10,23:56:00,1.06160
1437,0,11,23:57:00,1.16776
1438,0,10,23:58:00,1.06160


In [14]:
#3월 14일 부터 오늘까지 데이터 csv로 저장
startTime = pd.datetime(year = 2022, month = 3, day = 14)
endTime = pd.datetime.today().date() - datetime.timedelta(days=1)

C:\Users\sjyoo\Anaconda3\envs\fitbit\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  """Entry point for launching an IPython kernel.
C:\Users\sjyoo\Anaconda3\envs\fitbit\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  


In [16]:
date_list = []
df_list = []
allDates = pd.date_range(start=startTime, end = endTime)

for oneDate in allDates:
    
    oneDate = oneDate.date().strftime("%Y-%m-%d")
    
    oneDayData = auth2_client.intraday_time_series('activities/heart', base_date=oneDate, detail_level='1sec')

    df = pd.DataFrame(oneDayData['activities-heart-intraday']['dataset'])
    
    date_list.append(oneDate)
    
    df_list.append(df)
    
final_df_list = []

for date, df in zip(date_list, df_list):

    if len(df) == 0:
        continue
    
    df.loc[:, 'date'] = pd.to_datetime(date)
    
    final_df_list.append(df)

final_df = pd.concat(final_df_list, axis = 0)

In [17]:
final_df

,time,value,date
0,16:46:38,73,2022-03-14
1,16:46:43,79,2022-03-14
2,16:46:48,80,2022-03-14
3,16:46:53,85,2022-03-14
4,16:47:03,84,2022-03-14
...,...,...,...
11368,23:58:26,86,2022-03-28
11369,23:58:36,84,2022-03-28
11370,23:58:41,83,2022-03-28
11371,23:58:46,82,2022-03-28


In [18]:
#csv로 저장
filename = 'all_intradata_1'
final_df.to_csv(filename + '.csv', index = False)